In [1]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!nvidia-smi 

Wed Aug 24 23:39:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchinfo import summary

In [4]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [5]:
# MNIST dataset
# pre-processor
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])

train_dataset = torchvision.datasets.MNIST(root='data/',
                                           train=True, 
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/',
                                          train=False, 
                                          transform=transform)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True, num_workers=2)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False, num_workers=2)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding='same'),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc1 = nn.Linear(7*7*32, 480)
        self.fc2   = nn.Linear(480, 120)
        self.fc3   = nn.Linear(120, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
summary(model, input_size=(batch_size, 1,28, 28))

Layer (type:depth-idx)                   Output Shape              Param #
ConvNet                                  [100, 10]                 --
├─Sequential: 1-1                        [100, 16, 14, 14]         --
│    └─Conv2d: 2-1                       [100, 16, 28, 28]         416
│    └─BatchNorm2d: 2-2                  [100, 16, 28, 28]         32
│    └─ReLU: 2-3                         [100, 16, 28, 28]         --
│    └─MaxPool2d: 2-4                    [100, 16, 14, 14]         --
├─Sequential: 1-2                        [100, 32, 7, 7]           --
│    └─Conv2d: 2-5                       [100, 32, 14, 14]         12,832
│    └─BatchNorm2d: 2-6                  [100, 32, 14, 14]         64
│    └─ReLU: 2-7                         [100, 32, 14, 14]         --
│    └─MaxPool2d: 2-8                    [100, 32, 7, 7]           --
├─Linear: 1-3                            [100, 480]                753,120
├─Linear: 1-4                            [100, 120]                57,720
├

In [8]:

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.0855
Epoch [1/5], Step [200/600], Loss: 0.0775
Epoch [1/5], Step [300/600], Loss: 0.1098
Epoch [1/5], Step [400/600], Loss: 0.0338
Epoch [1/5], Step [500/600], Loss: 0.0572
Epoch [1/5], Step [600/600], Loss: 0.0361
Epoch [2/5], Step [100/600], Loss: 0.0603
Epoch [2/5], Step [200/600], Loss: 0.0320
Epoch [2/5], Step [300/600], Loss: 0.0426
Epoch [2/5], Step [400/600], Loss: 0.0437
Epoch [2/5], Step [500/600], Loss: 0.0305
Epoch [2/5], Step [600/600], Loss: 0.1146
Epoch [3/5], Step [100/600], Loss: 0.0091
Epoch [3/5], Step [200/600], Loss: 0.0144
Epoch [3/5], Step [300/600], Loss: 0.0034
Epoch [3/5], Step [400/600], Loss: 0.0516
Epoch [3/5], Step [500/600], Loss: 0.0061
Epoch [3/5], Step [600/600], Loss: 0.0548
Epoch [4/5], Step [100/600], Loss: 0.0051
Epoch [4/5], Step [200/600], Loss: 0.0271
Epoch [4/5], Step [300/600], Loss: 0.0212
Epoch [4/5], Step [400/600], Loss: 0.0516
Epoch [4/5], Step [500/600], Loss: 0.0402
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Test the model
import time


model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)

start_time = time.time()
with torch.no_grad():
    correct = 0
    total = 0
    pred =[]
    y_true = []
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
        pred+=predicted.tolist()
        y_true+=labels.tolist()

#     print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.7258899211883545 seconds ---


In [10]:
from sklearn.metrics import classification_report
print(classification_report(y_true, pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      1.00      0.99      1032
           3       0.99      0.99      0.99      1010
           4       0.98      1.00      0.99       982
           5       0.99      0.98      0.99       892
           6       0.99      0.99      0.99       958
           7       0.99      0.99      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.97      0.98      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



In [11]:
# import torch.autograd.profiler as profiler
# with profiler.profile(with_stack=True, profile_memory=True) as prof:
#     out = model(images)

# print(prof.key_averages(group_by_stack_n=5).table(sort_by='self_cpu_time_total', row_limit=5))